# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import regex as re
import emoji
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import seaborn as sns
import csv
from chardet import detect
from spellchecker import SpellChecker
nltk.download('wordnet')
nltk.download('punkt')

## Loading all Datasets

In [ ]:
dataset_pre = pd.read_csv('Datasets/PreElection.csv')

In [ ]:
dataset_dr = pd.read_csv('Datasets/DuringElection.csv')

In [ ]:
dataset_post = pd.read_csv('Datasets/PostElection.csv')

# Cleaning Datasets

# Add Labels